In [1]:
import scipy
import numpy as np
from pyparsing import col

from util.data_loader import DataLoader

dataset = DataLoader().main()
user_movie_matrix = dataset.train.pivot(index="user_id", columns="movie_id", values="rating")

user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))
matrix = user_movie_matrix.fillna(
    dataset.train.rating.mean()
).to_numpy()

P,S,Qt = scipy.sparse.linalg.svds(matrix, k=5)

pred_matrix = np.dot(np.dot(P,np.diag(S)), Qt)

print(f"P:{P.shape}, S:{S.shape}, Qt:{Qt.shape}, pred_matrix: {pred_matrix.shape}")

C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv(os.path.join(self.data_path,"movies.dat"), encoding="latin-1", names=cols,
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tags = pd.read_csv(os.path.join(self.data_path, "tags.dat"), encoding='latin-1',
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as re

P:(1000, 5), S:(5,), Qt:(5, 6669), pred_matrix: (1000, 6669)


In [11]:
from util.models import RecommendResult, Dataset
from base_recommender import BaseRecommender
from collections import defaultdict
import scipy
import numpy as np
np.random.seed(0)
import pandas as pd

class SVDRecommender(BaseRecommender):
    def recommend(self, dataset, **kwargs)->RecommendResult:
        fillna_with_zero = kwargs.get("fillna_with_zero", True)
        factors = kwargs.get("factors",5)
        #유저의 평점 예측
        user_movie_matrix = dataset.train.pivot(index="user_id",columns="movie_id",values="rating")

        if fillna_with_zero:
            matrix = user_movie_matrix.fillna(0).to_numpy()
        else:
            matrix = user_movie_matrix.fillna(dataset.train.rating.mean()).to_numpy()

        s,v,d = scipy.sparse.linalg.svds(matrix,k=factors)
        pred_matrix = np.dot(np.dot(s,np.diag(v)),d)
        #내가 추가한거
        pred_matrix = pd.DataFrame(pred_matrix, index=user_movie_matrix.index, columns = user_movie_matrix.columns)

        average_score = dataset.train.rating.mean()
        test = dataset.test.copy()
        pred_results = []
        train_users = dataset.train.user_id.unique()
        train_movies= dataset.train.movie_id.unique()
        for i, row in dataset.test.iterrows():
            user_id = row["user_id"]
            movie_id = row["movie_id"]
            if user_id not in train_users or movie_id not in train_movies:
                pred_results.append(average_score)
                continue

            pred_score = pred_matrix.loc[user_id, movie_id]
            pred_results.append(pred_score)
        test["rating_pred"]=pred_results

        #유저가 좋아할만한 아이템 10개 추천
        pred_love_items = defaultdict(list)
        user_evaluated_movies = dataset.train.groupby("user_id").agg({"movie_id":list})["movie_id"].to_dict()

        for user_id in dataset.train.user_id.unique():

            sorted_values = np.argsort(-pred_matrix.loc[user_id].values)
            movie_ids =  pred_matrix.columns[sorted_values]
            for movie_id in movie_ids:

                if movie_id not in user_evaluated_movies[user_id]:
                    pred_love_items[user_id].append(movie_id)
                if len(pred_love_items[user_id])==10:
                    break
        return RecommendResult(test.rating_pred, pred_love_items)

if __name__ == "__main__":
    SVDRecommender().eval()

C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv(os.path.join(self.data_path,"movies.dat"), encoding="latin-1", names=cols,
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tags = pd.read_csv(os.path.join(self.data_path, "tags.dat"), encoding='latin-1',
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as re

rmse: 3.331, recall: 0.078, precision:0.024
